In [1]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment, Environment, Model
from azureml.core.compute import ComputeTarget

In [2]:
from azureml.core.runconfig import RunConfiguration, CondaDependencies, DEFAULT_CPU_IMAGE
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PublishedPipeline
from azureml.pipeline.core import StepSequence
from azureml.widgets import RunDetails

In [3]:
import pandas as pd
import numpy as np
import os
import random as r

In [4]:
ws = Workspace.from_config()

In [5]:
compute_name = 'compute1548'
compute_target = ComputeTarget(ws, compute_name)

In [6]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [7]:
sepal_length_range = np.arange(4.3, 7.9, 0.1)
sepal_width_range = np.arange(2, 4.4, 0.1)
petal_length_range = np.arange(1, 6.9, 0.1)
petal_width_range = np.arange(0.1, 2.5, 0.1)
IrisList = []
columns = ['sepal_length','sepal_width','petal_length','petal_width']
IrisDF = pd.DataFrame(columns=columns)
for i in range(0,100):
    values = [round(r.choice(sepal_length_range),1),round(r.choice(sepal_width_range),1),round(r.choice(petal_length_range),1),round(r.choice(petal_width_range),1)]
    iris_dictionary = pd.DataFrame(dict(zip(columns, values)),index=[0])
    IrisList.append(iris_dictionary)
IrisDF = IrisDF.append(IrisList,True)
IrisDF.to_csv('Iris_Scoring_Data_for_ADF.csv')

In [8]:
Dataset.Tabular.register_pandas_dataframe(IrisDF,datastore,'Iris Scoring')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/5969d497-d912-4a53-ac9f-21aa2b31d048/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/5969d497-d912-4a53-ac9f-21aa2b31d048/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "54046df9-da88-41e0-9185-5a6294383e96",
    "name": "Iris Scoring",
    "version": 2,
    "workspace": "Workspace.create(name='mlws-final', subscription_id='80b2bae0-9f97-4190-a0d1-f57c078442d8', resource_group='work')"
  }
}

In [9]:
os.makedirs('Scoring_Scripts', exist_ok=True)  

In [10]:
%%writefile Scoring_Scripts/Iris_Scoring.py
from azureml.core import Run, Workspace
from azureml.core import Dataset, Datastore, Model

import os
import joblib
import numpy as np
import pandas as pd

run = Run.get_context()

def main():
    ws = run.experiment.workspace
    datastore = Datastore.get_default(ws)
    model_path = Model.get_model_path('Iris-Multi-Classification-AutoML')
    model = joblib.load(model_path)
    dataset = Dataset.get_by_name(ws,'Iris Scoring')
    scoringDF = dataset.to_pandas_dataframe()
    predictions = model.predict(scoringDF)
    predSeries = pd.Series(predictions)
    scoringDF['Prediction'] = predSeries

    output_datastore_path = 'Output_Folder'
    os.makedirs(output_datastore_path, exist_ok=True) 
    FileName = "Iris_Predictions.csv"
    OutputPath = os.path.join(output_datastore_path, FileName)
    scoringDF.to_csv(OutputPath, index = False, sep=',')
    datastore.upload_files(files=[OutputPath], target_path=output_datastore_path, overwrite=True)
    os.remove(OutputPath)
    os.rmdir(output_datastore_path)
    
if __name__ == '__main__':
    main()

Overwriting Scoring_Scripts/Iris_Scoring.py


In [11]:
# Create an Environment (collection of Python packages) you will use to run the script
Env = Environment(name='AutoML Environment')

# Create a CondaDepencies variable
conda_dep = CondaDependencies()

# Add conda packages
conda_dep.add_conda_package("numpy==1.18.5")
conda_dep.add_conda_package("joblib==0.14.1")
conda_dep.add_conda_package("pandas==0.25.3")
conda_dep.add_conda_package("packaging==20.7")
conda_dep.add_conda_package("xgboost==0.90")

# Add pip packages
conda_dep.add_pip_package("azureml-defaults==1.19.0")
conda_dep.add_pip_package("azureml-automl-core==1.19.0")
conda_dep.add_pip_package("azureml-automl-runtime==1.19.0")

# Adds dependencies to PythonSection of myenv
Env.python.conda_dependencies=conda_dep

# Register the environment to your workspace
RegisteredEnvironment = Env.register(workspace=ws)

In [12]:
run_config = RunConfiguration()
run_config.environment = Env
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [13]:
scoring_step = PythonScriptStep(name='iris-scoring-step',
script_name='Iris_Scoring.py',
source_directory='Scoring_Scripts', 
arguments=[],
inputs=[],
compute_target=compute_target,
runconfig=run_config,
allow_reuse=False)


In [14]:
# Create the training (Estimator) step by filling in all of the parameters
scoring_step = PythonScriptStep(name='iris-scoring-step',
                                         script_name='Iris_Scoring.py',
                                         source_directory='Scoring_Scripts',
                                         arguments=[],
                                         inputs=[], 
                                         compute_target=compute_target,
                                         runconfig=run_config,
                                         allow_reuse=False)

In [15]:
step_sequence = StepSequence(steps=[scoring_step])
pipeline = Pipeline(workspace=ws, steps=step_sequence)

In [16]:
# Run your pipeline
pipeline_experiment = Experiment(ws, 'Iris-Scoring-Pipeline-Run')
pipeline_run = pipeline_experiment.submit(pipeline, show_output=True)

Created step iris-scoring-step [9e522269][f7c6b77c-b0bd-48c3-a7e6-a9e595c5ef44], (This step will run and generate new outputs)
Submitted PipelineRun ffcd1c67-3d8b-4029-a7a2-e175625fdca5
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ffcd1c67-3d8b-4029-a7a2-e175625fdca5?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694


In [17]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: ffcd1c67-3d8b-4029-a7a2-e175625fdca5
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ffcd1c67-3d8b-4029-a7a2-e175625fdca5?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694
PipelineRun Status: Running


StepRunId: 12e201c6-6532-4256-80f0-32ba880e9517
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/12e201c6-6532-4256-80f0-32ba880e9517?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694
StepRun( iris-scoring-step ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_e1d785f2f7a4113e996c389fbaed0735d7730c38a5956a6d9eebdaf3f9d04a7c_d.txt
2021-09-06T14:51:23Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/mlws-final/azureml/12e201c6-6532-4256-80f0-32ba880e9517/mounts/workspaceblobstore --tmp-path=/mnt/batch/tas

'Finished'

In [18]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Iris-Scoring-Pipeline',\
    description='Pipeline that Scores Iris Data', version= '1.0')

published_pipeline

Name,Id,Status,Endpoint
Iris-Scoring-Pipeline,fd6221aa-1b28-4e51-8490-352c7c1cb1f1,Active,REST Endpoint
